In [ ]:
% bigquery 
-- 1. CONFIGURATION
DECLARE analysis_start TIMESTAMP DEFAULT TIMESTAMP("2024-01-15 00:00:00 UTC");
DECLARE analysis_end   TIMESTAMP DEFAULT TIMESTAMP("2024-01-18 00:00:00 UTC");

-- Logic Settings
DECLARE lookahead_blocks INT64 DEFAULT 600; 
DECLARE prefix_len INT64 DEFAULT 3;
DECLARE suffix_len INT64 DEFAULT 4;
DECLARE dust_threshold FLOAT64 DEFAULT 1.0; -- Upper limit for a "Tiny" transfer ($1.00)

DECLARE start_block DEFAULT (
  SELECT MIN(block_number)
  FROM `bigquery-public-data.goog_blockchain_optimism_mainnet_us.logs`
  WHERE block_timestamp >= analysis_start
);

DECLARE end_block DEFAULT (
  SELECT MAX(block_number)
  FROM `bigquery-public-data.goog_blockchain_optimism_mainnet_us.logs`
  WHERE block_timestamp < analysis_end
);

CREATE TEMP FUNCTION HexToBigNumeric(hex STRING)
RETURNS BIGNUMERIC
LANGUAGE js AS """
  // Remove 0x prefix if present
  hex = hex.startsWith("0x") ? hex.slice(2) : hex;

  // Convert hex to BigInt
  const value = BigInt('0x' + hex);

  // Return as string so BigQuery can cast it
  return value.toString();
""";

-- 2. DATA EXTRACTION
WITH raw_transfers AS (
    SELECT
    block_number,
    block_timestamp,
    transaction_hash,
    address AS token_address,
    -- Decode from & to (topics[1] and topics[2])
    CONCAT("0x", SUBSTR(topics[SAFE_OFFSET(1)], 27)) AS from_addr,
    CONCAT("0x", SUBSTR(topics[SAFE_OFFSET(2)], 27)) AS to_addr,
    -- cast from amounts
    HexToBigNumeric(data) AS value,
    HexToBigNumeric(data) / 1e6 AS value_usd,
    -- add token symbol manually
    -- CASE 
    --     WHEN LOWER(address) = usdc_native  THEN "USDC.e"
    --     WHEN LOWER(address) = usdc_bridged THEN "USDC"
    --     WHEN LOWER(address) = usdt_addr    THEN "USDT"
    -- END AS token_symbol
    FROM `bigquery-public-data.goog_blockchain_optimism_mainnet_us.logs`
    WHERE block_number BETWEEN start_block AND end_block
    AND removed = FALSE
    AND topics[SAFE_OFFSET(0)] = "0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef"
    AND address IN (
        "0x0b2c639c533813f4aa9d7837caf62653d097ff85", -- USDC Native
        "0x7f5c764cbc14f9669b88837ca1490cca17c31607", -- USDC Bridged
        "0x94b008aa00579c1307b0ef2c499ad98a8ce58e58"  -- USDT
    )
),

-- 3. IDENTIFY PAIRS
attack_pairs AS (
    SELECT
        S.transaction_hash AS attack_tx_hash,
        S.block_number AS attack_block,
        S.block_timestamp AS attack_ts,
        -- S.token_symbol,
        S.value_usd,
        
        -- NEW COLUMN: Attack Type
        CASE 
            WHEN S.value_usd = 0 THEN 'zero_value'
            ELSE 'tiny_transfer'
        END AS attack_type,

        -- Attacker Labeling
        CASE 
            WHEN S.from_addr = V.from_addr THEN S.to_addr 
            ELSE S.from_addr 
        END AS attacker_address,

        V.from_addr AS victim_address,
        V.to_addr AS intended_address,
        V.transaction_hash AS intended_tx_hash,
        V.block_timestamp AS intended_ts,
        V.block_number AS intended_block,
        
        CASE WHEN S.from_addr = V.from_addr THEN 'spoof_outgoing' ELSE 'spam_incoming' END AS direction

    FROM raw_transfers AS V -- History
    JOIN raw_transfers AS S -- Attack
        ON S.token_address = V.token_address
        AND S.block_number >= V.block_number
        AND S.block_number <= (V.block_number + lookahead_blocks)
    WHERE
        V.value_usd > 0
        AND V.block_timestamp BETWEEN analysis_start AND analysis_end
        
        -- MODIFIED FILTER: Allow 0 OR Small Values
        AND S.value_usd <= dust_threshold
        
        AND (
            (S.from_addr = V.from_addr)
            OR
            (S.to_addr = V.from_addr)
        )
        AND S.transaction_hash != V.transaction_hash
)

-- 4. FINAL SELECTION
SELECT 
    attack_tx_hash,
    attack_block,
    attack_ts,
    -- token_symbol,
    attack_type, -- <--- Included in final output
    value_usd,   -- <--- Good to verify the actual amount
    direction,
    attacker_address,
    victim_address,
    intended_address,
    intended_ts,
    (attack_block - intended_block) AS blocks_delay
FROM attack_pairs
WHERE
    SUBSTR(attacker_address, 3, prefix_len) = SUBSTR(intended_address, 3, prefix_len)
    AND
    SUBSTR(attacker_address, -1 * suffix_len) = SUBSTR(intended_address, -1 * suffix_len)
    AND attacker_address != intended_address

-- DEDUPLICATION
QUALIFY ROW_NUMBER() OVER(
    PARTITION BY attack_tx_hash, attacker_address, victim_address, direction
    ORDER BY intended_ts DESC
) = 1
ORDER BY attack_ts, attack_tx_hash